In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import scipy
from timeit import default_timer as timer
from datetime import timedelta

from concurrent.futures import ThreadPoolExecutor
import threading

movies = pd.read_csv('movie_archive\movies_metadata.csv')
ratings_small = pd.read_csv('movie_archive\\ratings_small.csv')
links_small = pd.read_csv('movie_archive\links_small.csv')

C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
ratings_train, ratings_test = train_test_split(ratings_small, test_size=0.3)


In [3]:
print(len(ratings_train))
print(len(ratings_test))

70002
30002


In [4]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), stop_words='english', max_features= 1000000)
tfidf_matrix = tfidf.fit_transform(movies['title'].apply(str) + " " + movies['overview'].apply(str))
#tfidf_matrix = tfidf.fit_transform(movies['overview'].apply(str))
tfidf_matrix

<45466x1000000 sparse matrix of type '<class 'numpy.float64'>'
	with 2536808 stored elements in Compressed Sparse Row format>

In [5]:
ratings_train_merge = ratings_train.merge(links_small, on='movieId')
ratings_train_merge.head()
ratings_test_merge = ratings_test.merge(links_small, on='movieId')
ratings_test_merge.head()

,userId,movieId,rating,timestamp,imdbId,tmdbId
0,633,587,3.0,848518496,99653,251.0
1,388,587,4.0,946523761,99653,251.0
2,67,587,3.0,854714282,99653,251.0
3,605,587,3.0,980177887,99653,251.0
4,30,587,2.0,945277405,99653,251.0


In [6]:
# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}
count_to_big = 0
count_false_int = 0

# get all movie vector rated from user
def get_user_rated_movies(ids):
    
        movie_vector_list = []
        for i in ids:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
            try:
                index = movieid_list.index(int(tmdbId))
                movie_vector = tfidf_matrix[index]
                movie_vector_list.append(movie_vector)
            except ValueError:
                #print ("ValueError tmdbId: " + str(tmdbId))     # deleted/NaN movies
                count_false_int = count_false_int + 1


        movie_vectors = scipy.sparse.vstack(movie_vector_list)

        return movie_vectors


def create_user_vector_jit(user_rated_movies_vector_array):
        
        user_vector = user_rated_movies_vector_array.sum(axis=0)

        return user_vector


def create_user_vectors(uid):
        
        # --- Train Data ---
        user_profile = ratings_train.loc[ratings_train['userId'] == uid]
            
        user_rated_movies_vector_list = get_user_rated_movies(user_profile['movieId'].to_list())
        n = user_rated_movies_vector_list.shape[0]

        if n > 1500:
            count_to_big = count_to_big + 1

        else:
            
            user_vectors_array = scipy.sparse.csr_matrix.toarray(user_rated_movies_vector_list)
            user_vectors_array.reshape(user_rated_movies_vector_list.shape)

            user_vector = create_user_vector_jit(user_vectors_array)

            user_norm = sklearn.preprocessing.normalize(scipy.sparse.csr_matrix(user_vector))
            all_user_profiles[uid] = user_norm


# Sorting Threading
recommendation_users = {}

def sort_user_vectors(uid):
    if uid in all_user_profiles.keys():
        user_vec = all_user_profiles[uid]
        cosine_similarity_user_movies = cosine_similarity(user_vec, tfidf_matrix)
        top_similar = cosine_similarity_user_movies[0].argsort()[:100:-1]
        #top_similar = cosine_similarity_user_movies[0].sort(kind='timsort')[:100:-1]
        #top_similar = np.sort(cosine_similarity_user_movies[0], kind='stable')
        recommendation_users[uid] = [(cosine_similarity_user_movies[0][i], movies['id'][i]) for i in top_similar]

In [7]:
all_user_profiles = {}
count_to_big = 0
count_false_int = 0
recommendation_users = {}

userid_list = ratings_train['userId'].unique()
movieid_list = movies['id'].to_list()
# changes movieId type to int and deletes all false entrys
movieid_list = [x for x in movieid_list if x.isdigit()]
movieid_list = list(map(int, movieid_list))

start = timer()

with ThreadPoolExecutor(max_workers=8) as executer:
    executer.map(create_user_vectors, userid_list)

end = timer()
print ("Time Calculate Vectors HH:MM:SS: ",timedelta(seconds=end-start))


start = timer()

with ThreadPoolExecutor(max_workers=8) as executer:
    executer.map(sort_user_vectors, userid_list)

end = timer()
print ("Time Sorting HH:MM:SS: ",timedelta(seconds=end-start))





Time Calculate Vectors HH:MM:SS:  0:02:08.609323
Time Sorting HH:MM:SS:  0:02:58.636420


In [8]:
recommendation_users_top = recommendation_users
for k, v in recommendation_users_top.items():
    recommendation_users_top[k] = v[:10000]

len(recommendation_users_top[3])

10000

In [9]:
# Remove movies used to create User Vector

percentage_already_seen = []
cleaned_recommendations = {}
for uid in userid_list:
    if uid in all_user_profiles.keys():
        user_profile = ratings_train.loc[ratings_train['userId'] == uid]
        user_recommendation = recommendation_users_top[uid]

        user_profile_mid_list = user_profile['movieId'].to_list()
        user_tmdb_list = []
        for m in user_profile_mid_list:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
            try:
                user_tmdb_list.append(int(tmdbId))
            except ValueError:
                #print("ValueError: " + str(tmdbId))
                count_false_int = count_false_int + 1
            
        m_count = 0
        for cos, mid in user_recommendation:
            try:
                if int(mid) in user_tmdb_list:
                    m_count += 1
                    user_recommendation.remove((cos,mid))
            except ValueError:
                print("ValueError: " + str(mid) + " not in tmdbid list")    #Fehler weil, schon gesucht aber nicht gefunden // skipped because to big sample size

        cleaned_recommendations[uid] = user_recommendation


ValueError: 2014-01-01 not in tmdbid list
ValueError: 1997-08-20 not in tmdbid list
ValueError: 2012-09-29 not in tmdbid list
ValueError: 2012-09-29 not in tmdbid list
ValueError: 2014-01-01 not in tmdbid list
ValueError: 1997-08-20 not in tmdbid list
ValueError: 2012-09-29 not in tmdbid list
ValueError: 1997-08-20 not in tmdbid list
ValueError: 2014-01-01 not in tmdbid list
ValueError: 2014-01-01 not in tmdbid list
ValueError: 2012-09-29 not in tmdbid list
ValueError: 2012-09-29 not in tmdbid list
ValueError: 1997-08-20 not in tmdbid list
ValueError: 2012-09-29 not in tmdbid list
ValueError: 1997-08-20 not in tmdbid list
ValueError: 2014-01-01 not in tmdbid list
ValueError: 1997-08-20 not in tmdbid list
ValueError: 2012-09-29 not in tmdbid list
ValueError: 2014-01-01 not in tmdbid list
ValueError: 1997-08-20 not in tmdbid list
ValueError: 2012-09-29 not in tmdbid list
ValueError: 2014-01-01 not in tmdbid list
ValueError: 1997-08-20 not in tmdbid list
ValueError: 2014-01-01 not in tmdb

In [10]:
# Calculate Percentage of user rated movies from test split in recommended, top 10/50/100
top_value = 100
user_percentages = []
nan_count = 0
for uid in userid_list:
    user_profile = ratings_test.loc[ratings_test['userId'] == uid]
    found_movies = 0
    if uid in recommendation_users.keys():
        user_recommendations = cleaned_recommendations[uid]
        #print(len(user_recommendations))
        #tmdbId_list = user_profile['tmdbId'].to_list():

        user_tmdb_list = []
        for m in user_profile_mid_list:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
            try:
                user_tmdb_list.append(int(tmdbId))
            except ValueError:
                #print("ValueError: " + str(tmdbId))
                count_false_int = count_false_int + 1

        for cos, mid in user_recommendations[:top_value]:
            try:
                if int(mid) in user_tmdb_list:
                    found_movies = found_movies + 1
            except ValueError:
                print("ValueError: " + str(mid) + " not in tmdbid list")

       
    user_percentages.append(found_movies/len(user_profile))

mean_percentage = sum(user_percentages)/len(user_percentages)
print("Mean Percentage of good Recommendations (based on Test Split): " + str(mean_percentage))

ValueError: 2012-09-29 not in tmdbid list
ValueError: 1997-08-20 not in tmdbid list
ValueError: 2014-01-01 not in tmdbid list
ValueError: 1997-08-20 not in tmdbid list
ValueError: 2014-01-01 not in tmdbid list
ValueError: 2012-09-29 not in tmdbid list
ValueError: 2014-01-01 not in tmdbid list
ValueError: 2012-09-29 not in tmdbid list
ValueError: 1997-08-20 not in tmdbid list
ValueError: 2012-09-29 not in tmdbid list
ValueError: 2014-01-01 not in tmdbid list
ValueError: 1997-08-20 not in tmdbid list
ValueError: 2014-01-01 not in tmdbid list
ValueError: 1997-08-20 not in tmdbid list
ValueError: 2012-09-29 not in tmdbid list
ValueError: 2012-09-29 not in tmdbid list
ValueError: 1997-08-20 not in tmdbid list
ValueError: 2014-01-01 not in tmdbid list
Mean Percentage of good Recommendations (based on Test Split): 0.026179622284996623


In [ ]:
# feature size 1mio, title + overview, top100
Mean Percentage of good Recommendations (based on Test Split): 0.06530752814292558

# feature size 1mio, only overview, top100
Mean Percentage of good Recommendations (based on Test Split): 0.022035657968376067